In [1]:
import numpy as np

In [17]:
class ErrorEstimator:
    def __init__(self, distribution, uncertanties, niter=10000, seed=None):
        self.distr, self.uncs = list(self.check_np(distribution, uncertanties))
        assert self.distr.shape == self.uncs.shape, 'Distribution and unceratinties must be same length,' \
                                                    + f' but {self.distr.shape} and {self.uncs.shape} given'
        self.norm()
        self.niter = niter
        self.rng = np.random.default_rng(seed=seed)
        self.bins = np.array(list(range(self.distr.size)))

    def estimate_uncertanties(self, to_print=False):
        x = np.empty([self.niter, self.distr.size])
        for i, [m, s] in enumerate(zip(self.distr, self.uncs)):
            x[:, i] = self.rng.normal(loc=m, scale=s, size=self.niter)

        means = np.apply_along_axis(self.mean, 1, x)
        vs = np.apply_along_axis(self.variance, 1, x)

        result = {
            'mean': means.mean(),
            'mean_unc': means.std(),
            'var': vs.mean(),
            'var_unc': vs.std()
        }
        if to_print:
            pr = f"Mean: {result['mean']:.2f}+-{result['mean_unc']:.2f}\n" \
                 + f"Variance: {result['var']:.2f}+-{result['var_unc']:.2f}"
            print(pr)
            return
        return result

    def mean(self, d):
        return d @ self.bins

    def variance(self, d):
        sbins = [i * i for i in self.bins]
        return d @ sbins - (d @ self.bins) ** 2
    
    def norm(self):
        if np.allclose(self.distr.sum(), 1):
            return
        denom = self.distr.sum()
        self.distr = self.distr / denom
        self.uncs = self.uncs / denom      

    def check_np(self, *args):
        for term in args:
            if not isinstance(term, np.ndarray):
                yield np.asarray(term)
            else:
                yield term

### Input data:
- d: distribution - emittion probabilities distribution that will be normed if it doesn't. 
- u: uncertanties - unceratinties of the distribution, will be normed as well. 

In [14]:
d = [0.009147, 0.072394, 0.237469, 0.341113, 0.236059, 0.080930, 0.017569, 0.004882, 0, 0.000437]
u = [0.00069, 0.00191, 0.00285, 0.00308, 0.00253, 0.00170, 0.00113, 0.00080, 0.00042, 0.00010]

In [18]:
ErrorEstimator(d, u).estimate_uncertanties(to_print=True)

Mean: 3.06+-0.02
Variance: 1.42+-0.05
